In [1]:
import gc
import json
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm

from utils import loadpkl, removeCorrelatedVariables, removeMissingVariables

%matplotlib inline

In [2]:
files = sorted(glob('../features/*.feather'))
df = pd.concat([pd.read_feather(f) for f in tqdm(files, mininterval=60)], axis=1)

# set card_id as index
df.set_index('card_id', inplace=True)
df.head()

  0%|          | 0/1079 [00:00<?, ?it/s]/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pandas/io/feather_format.py:124: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  nthreads=int_use_threads)
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
100%|██████████| 1079/1079 [00:02<00:00, 524.14it/s]


,AOV,AOV_ratio,CLV,CLV_month_diff,CLV_month_diff_ratio,CLV_ratio,Mothers_Day_2018,amount_month_ratio_max,amount_month_ratio_mean,amount_month_ratio_min,...,weekofyear_9,year_0,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,182.318395,1.661881,4.690294e+06,1.705895e+08,-0.008053,0.013005,49.492977,NaN,NaN,-350.00,...,0,0,0,0,0,0,0,0,1,0
C_ID_3d0044924f,101.445060,0.159410,1.071890e+07,1.005798e+07,0.000050,0.000047,56.833333,NaN,NaN,7.99,...,0,0,0,0,0,0,0,0,1,0
C_ID_d639edf6cd,76.651163,0.679063,8.444000e+04,NaN,NaN,0.000367,15.720930,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
C_ID_186d6a6901,118.311688,0.856153,3.805900e+05,NaN,NaN,0.007076,55.623377,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
C_ID_cdbd2c0db2,249.208020,1.067407,2.299008e+06,-9.753155e+07,0.031752,0.078204,61.856099,NaN,NaN,-156.09,...,0,0,0,0,0,0,0,0,1,0


In [27]:
# https://www.kaggle.com/raddar/card-id-loyalty-different-points-in-time
df['hist_observation_date']= df.apply(lambda x: pd.to_datetime(x['hist_purchase_date_max']*10**9)-pd.DateOffset(months=x['hist_month_lag_max']), axis=1)
df['new_observation_date']=df.apply(lambda x: np.nan if np.isnan(x['new_month_lag_min']) else pd.to_datetime(x['new_purchase_date_min']*10**9)-pd.DateOffset(months=x['new_month_lag_min']-1), axis=1)

df['hist_observation_date'] = df['hist_observation_date'].dt.to_period('M').dt.to_timestamp()+pd.DateOffset(months=1)
df['new_observation_date'] = df['new_observation_date'].dt.to_period('M').dt.to_timestamp()

In [26]:
df[:10000]

card_id
C_ID_92a2005557   2018-03-05 14:04:36
C_ID_3d0044924f   2018-02-01 17:07:54
C_ID_d639edf6cd   2018-03-28 17:43:11
C_ID_186d6a6901   2018-03-07 11:55:06
C_ID_cdbd2c0db2   2018-03-02 11:55:43
C_ID_0894217f2f   2017-08-15 10:17:55
C_ID_7e63323c00   2018-03-06 12:37:14
C_ID_dfa21fc124   2018-01-06 07:03:41
C_ID_fe0fdac8ea   2018-03-17 14:22:31
C_ID_bf62c0b49d   2018-03-01 09:22:06
C_ID_92853cdb2c   2018-03-29 17:01:43
C_ID_269d816788   2018-03-12 05:50:28
C_ID_61d50d7057   2018-03-23 18:35:19
C_ID_4e07413433   2017-11-10 09:33:14
C_ID_b6302b31c6   2017-11-05 05:23:32
C_ID_3b69154173                   NaT
C_ID_9feec11e78   2018-03-07 08:06:44
C_ID_f6658dbefe   2018-03-03 15:42:34
C_ID_4a7dda0f9e   2018-03-01 13:27:11
C_ID_6adae2a906   2018-03-03 11:11:38
C_ID_0b70ca7347   2018-03-12 16:12:43
C_ID_25b2509282                   NaT
C_ID_82fb11db22   2018-03-12 17:13:24
C_ID_e03db05bde   2018-03-02 10:24:07
C_ID_dcb7c76747   2018-03-09 07:31:00
C_ID_749cfc0c3a                   NaT
C_ID

In [ ]:
np.isnu